# How to style images using CycleGAN

In [1]:
from preprocessing.preprocessor import GANPreprocessor
from trainer.trainer_step import CycleGANTrainer
from zenml.datasources import ImageDatasource
from zenml.pipelines import TrainingPipeline
from zenml.repo import Repository
from zenml.steps.split import CategoricalDomainSplit

First we instantiate our current ZenML repository. That way, if you run this notebook
multiple times in succession, you will not run into trouble with errors on datasource
creation.

In [2]:
# Before running this, initialize the repository by calling `zenml init`
repo = Repository.get_instance()

### Creating the pipeline

Now we create the training pipeline that we will run our experiment with. We enable
caching by default, as it saves computation time and resources by saving results that do not have to
be recomputed over and over again, such as splits or preprocessing.

In [3]:
gan_pipeline = TrainingPipeline(name="gan_test", enable_cache=True)

2021-03-17 13:57:31,532 — zenml.pipelines.base_pipeline — INFO — Pipeline gan_test created.


### Creating the datasource

Next, we create the image datasource for the GAN. If you downloaded the data locally, and you want to run your pipeline
with the local copy of the data on your machine, run the `gan_images_ce.py` script located in your images folder
before executing the next cell. It will create a label file for use in your ZenML image source.

**Tip**: If you want to try out the GAN pipeline training on the full data set, you can set
`base_path="gs://zenml_quickstart/cycle_gan"` in the try-block in the cell right below to work with the full dataset.
For this demo, we focus on quick pipeline execution, so we do not use the data to its full extent.

In [4]:
try:
    base_path = "gs://zenml_quickstart/cycle_gan_mini"
    ds = ImageDatasource(name="gan_images", base_path=base_path)
except:
    ds = repo.get_datasource_by_name('gan_images')

2021-03-17 13:57:32,704 — zenml.datasources.base_datasource — INFO — Datasource gan_images created.


### Adding the datasource to the pipeline

The datasource is the first step in the pipeline, because it supplies the data on
which the model will be trained. You can add it to the pipeline as follows:

In [5]:
gan_pipeline.add_datasource(ds)

### Add a split

To prepare our images, we need to separate the real images from the styled ones (Monet
paintings in this case). Therefore, we add the image type (real or Monet) as a label
and split on that. The real images are going to be saved as evaluation data, because we
will use them later in image style generation.

In [6]:
gan_pipeline.add_split(CategoricalDomainSplit(categorical_column="label",
                                              split_map={"train": ["monet"],
                                                         "eval": ["real"]}))

### Add a preprocessing step

Next, we add a small preprocessing step for our image. It is defined in the
`preprocessing.py` file in this folder, and contains two separate steps: Loading the
image (up to this point, it is persisted as a binary string) and normalizing its values
 between -1 and 1.

In [7]:
gan_pipeline.add_preprocesser(GANPreprocessor())

### Add a training step

Now we come to what is the most involved part of this example - a custom trainer step.
It is defined in the `gan_functions.py` file in the `trainer` subfolder, and contains a Keras
implementation of the CycleGAN model architecture. Along with this, it also comes with
utilities for loading data, preparing it as `tf.data.Dataset`s.

In [8]:
gan_pipeline.add_trainer(CycleGANTrainer(epochs=1))

2021-03-17 13:57:35,594 — zenml.repo.git_wrapper — WARNING — Found uncommitted file. Pipelines run with this configuration may not be reproducible. Please commit all files in this module and then run the pipeline to ensure reproducibility.


### Run your pipeline
Now that all the required steps are in place, it is time to run the newly created
pipeline.

### OPTIONAL: Running the pipeline on Google Cloud Platform

If you want to run the pipeline on GCP instead, please skip the next cell and head
straight to the section afterwards.

In [9]:
gan_pipeline.run()

2021-03-17 13:57:36,587 — zenml.repo.git_wrapper — WARNING — Found uncommitted file. Pipelines run with this configuration may not be reproducible. Please commit all files in this module and then run the pipeline to ensure reproducibility.
2021-03-17 13:57:36,592 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component DataGen is running.


2021-03-17 13:57:36,894 — apache_beam.runners.interactive.interactive_environment — WARNING — Dependencies required for Interactive Beam PCollection visualization are not available, please use: `pip install apache-beam[interactive]` to install necessary dependencies to enable all data visualization features.


2021-03-17 13:57:40,249 — apache_beam.internal.gcp.auth — WARNING — Unable to find default credentials to use: The Application Default Credentials are not available. They are available if running in Google Compute Engine. Otherwise, the environment variable GOOGLE_APPLICATION_CREDENTIALS must be defined pointing to a file defining the credentials. See https://developers.google.com/accounts/docs/application-default-credentials for more information.
Connecting anonymously.
2021-03-17 13:57:48,814 — apache_beam.io.tfrecordio — WARNING — Couldn't find python-snappy so the implementation of _TFRecordUtil._masked_crc32c is not as fast as it could be.


2021-03-17 13:57:49,018 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component DataGen is finished.
2021-03-17 13:57:49,020 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component DataStatistics is running.
2021-03-17 13:57:50,215 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component DataStatistics is finished.
2021-03-17 13:57:50,222 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component DataSchema is running.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


2021-03-17 13:57:50,270 — tensorflow — WARNING — From /home/hamza/.virtualenvs/ce_opensource/lib/python3.6/site-packages/tensorflow_data_validation/utils/stats_util.py:247: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


...

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


2021-03-17 13:59:05,232 — tensorflow — WARNING — From /home/hamza/.virtualenvs/ce_opensource/lib/python3.6/site-packages/tensorflow/python/training/tracking/tracking.py:111: Layer.updates (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /home/hamza/workspace/maiot/github_temp/zenml/.zenml/local_store/c6f55900490949ef8d48b99eb7514ead/Trainer/model/120/serving_model_dir/assets


2021-03-17 13:59:22,485 — tensorflow — INFO — Assets written to: /home/hamza/workspace/maiot/github_temp/zenml/.zenml/local_store/c6f55900490949ef8d48b99eb7514ead/Trainer/model/120/serving_model_dir/assets


2021-03-17 13:59:23,375 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component Trainer is finished.


And that is it! If you studied the contents of the `gan_functions.py` file, you will
notice that there's also a custom callback function implemented. It uses TensorBoard
to display the same single, Monet-styled image generated by the CycleGAN after each
epoch of learning. There is a neat little slider that lets you look through all logged
epochs - that way, you can visualize the CycleGAN's style learning process. Let's try it out.

### Inspect your model's style evolution on TensorBoard

Now we launch a TensorBoard extension to inspect our model's training progress. With our default configuration, we
logged four different loss values, which measure the abilities of the sub–networks (Monet/real photo generators and
discriminators) to create realistic data and discriminate real from artificial images, respectively.

On top of that, we added a custom callback that logs a single image generated by the Monet Generator network from a
real photo. From that, we are able to see how the style changes throughout the training process.

In [11]:
# Executing this cell should spawn another cell below.
# #Running that will add a TensorBoard dashboard.

from zenml.utils.post_training.post_training_utils import get_tensorboard_block, create_new_cell
from zenml.enums import GDPComponent

tensorboard_root = gan_pipeline.get_artifacts_uri_by_component(GDPComponent.Trainer.name)[0]

create_new_cell(get_tensorboard_block(tensorboard_root))

In [12]:
import os

model_path = '"/home/hamza/workspace/maiot/github_temp/zenml/.zenml/local_store/c6f55900490949ef8d48b99eb7514ead/Trainer/model/120"'
logdir = os.path.join(model_path, 'logs')
%load_ext tensorboard
%tensorboard --logdir {logdir}

Now you should see the TensorBoard dashboard initialized above. Click on the "image" tab in the header to see how
the image evolves during training. As a next step if you are interested, you can increase the training time or tweak
some of the hyperparameters (learning rates, regularization) to see how the style changes.

## OPTIONAL: Running a training pipeline on Google Cloud Platform

ZenML also enables you to run your local configuration from above in the cloud
with the help of cloud-based orchestrator backend. This section will give you
an example of how to run a pipeline in the cloud.

First, we import all the additional necessary functions.

In [ ]:
from zenml.backends.orchestrator import OrchestratorGCPBackend
from zenml.metadata import MySQLMetadataStore
from zenml.repo import ArtifactStore

### Setting your cloud variables

In order for the cloud discovery to work, you will have to define some variables.
Among those are the CloudSQL server and artifact store locations, your local
metadata store, and your Google Cloud Project.

Please customize these variables below to match your own cloud settings and desired
configuration.

In [ ]:
# location of your cloud artifact store
artifact_store = 'gs://your-bucket-name/optional-subfolder'

# the GCP project to launch the VM in
project = 'PROJECT'
# the zone to launch the VM in
zone = 'europe-west1-b'

# your CloudSQL server configuration
cloudsql_connection_name = 'PROJECT:REGION:INSTANCE'

# your local metadata store setup
mysql_db = 'DATABASE'
mysql_user = 'USERNAME'
mysql_pw = 'PASSWORD'

### Creating backends, metadata store and artifact store

Now using the variables above, we can define our backends for orchestration and
training, as well as artifact and metadata stores.

In [ ]:
orchestrator_backend = OrchestratorGCPBackend(
    cloudsql_connection_name=cloudsql_connection_name,
    project=project,
    zone=zone)

metadata_store=MySQLMetadataStore(
        host='127.0.0.1',
        port=3306,
        database=mysql_db,
        username=mysql_user,
        password=mysql_pw)

artifact_store=ArtifactStore(artifact_store)

### Running the pipeline on Google Cloud

Now you can run your pipeline on your own Google Cloud project. Simply specify the
backends and custom artifact and metadata stores in your `pipeline.run()` call.

**NB**: The CycleGAN network in this tutorial is really large (in excess of a gigabyte
sometimes). If you run into problems with model deployment, it might be because of an
incompatibility on GCAIP's side with such large models.

An easy fix is to return the Monet Generator only in the CycleGANTrainer's `model_fn`
if you want to generate Monet-styled images from reals only, or the Real Generator if
you want to create real photos out of Monet paintings.

Both of these agents are available as class attributes of the CycleGAN, so just change
the return value of the `model_fn` to `return cycle_gan_model.{m_gen,p_gen}`
depending on which one you choose.

In [ ]:
# Run the pipeline on a Google Cloud VM
gan_pipeline.run(backend=orchestrator_backend,
                 metadata_store=metadata_store,
                 artifact_store=artifact_store)

Now you should have a successfully deployed model on your Google Cloud AI Platform
project.

# Requesting a deployed model in the Google Cloud

This next section assumes that you successfully ran a ZenML Pipeline on the CycleGAN
dataset, and that you deployed your model in the Google Cloud AI Platform.
We are now going to generate images by sending prediction requests to a deployed
Monet-styling CycleGAN model!

In [ ]:
# Imports and version checks
import tensorflow as tf

print(tf.__version__)

In [ ]:
import httplib2

print(httplib2.__version__)

Especially the httplib2 dependency is important here, since the prediction code
using the Google API Client requires a version of httplib2 that is rather new
(>= 0.16 should be fine). This example was run with version 0.18.1 of httplib2.

# Building a TFExample from an image and formatting it correctly

First, we point to an image and make a TFExample out of it. The conversion code looks like this:

In [ ]:
def prepare_tfexample(image_path):
    """
    Prepares an image into a TFExample understood by the served model.
    """
    with open(image_path, "rb") as img_file:
        real_image_raw = img_file.read()
        real_image = tf.io.decode_image(real_image_raw).numpy()

    feature = {
        "image": tf.train.Feature(bytes_list=tf.train.BytesList(value=[real_image_raw])),
    }

    # return the TFExample
    return tf.train.Example(features=tf.train.Features(feature=feature))

Now point to an example image from the CycleGAN dataset and convert it to a TFExample!

In [ ]:
# change this to a local image path (e.g. on your machine or on GCS)
image_path = "/path/to/sample_image.jpg"

example = prepare_tfexample(image_path)

In [ ]:
print(example)

#### Formatting

Looks good! Now we have to convert it to base64 so that AI Platform can parse the data properly.
More information can be found here: https://cloud.google.com/ai-platform/prediction/docs/online-predict

In [ ]:
import base64
instance = {"examples": {"b64": base64.b64encode(example.SerializeToString()).decode("utf-8")}}

# Requesting a prediction from our model

First we point the environment variable GOOGLE_APPLICATION_CREDENTIALS (used frequently by GCP) to our service account.

In [ ]:
import os

# change this to point to your service account file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/path/to/your/service_account.json"

### The actual prediction request code

The following code is used to query our model deployed in AI Platform. It can be found as well on the AI Platform Docs
under https://cloud.google.com/ai-platform/prediction/docs/online-predict.

In [ ]:
import googleapiclient
def predict_json(project, model, instances, version=None):
    """Send json data to a deployed model for prediction.

    Args:
        project (str): project where the AI Platform Model is deployed.
        model (str): model name.
        instances ([Mapping[str: Any]]): Keys should be the names of Tensors
            your deployed model expects as inputs. Values should be datatypes
            convertible to Tensors, or (potentially nested) lists of datatypes
            convertible to tensors.
        version: str, version of the model to target.
    Returns:
        Mapping[str: any]: dictionary of prediction results defined by the
            model.
    """
    # Create the AI Platform service object.
    # To authenticate set the environment variable
    # GOOGLE_APPLICATION_CREDENTIALS=<path_to_service_account_file>
    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

In [ ]:
# change this to your deployed model name
model_name = "YOUR_MODEL_NAME"

prediction = predict_json(project=project, model=model_name, instances=[instance])

And there we have it! Let us visualize the response right away, along with our original image to look at the style
transfer done by our model.

In [ ]:
import numpy as np
predicted_image = np.array(prediction[0]['output_0'])

predicted_image.shape

In [ ]:
with open(image_path, "rb") as img_file:
    real_image_raw = img_file.read()
    real_image = tf.io.decode_image(real_image_raw).numpy()

## Head-to-head comparison

In [ ]:
import matplotlib.pyplot as plt
_, ax = plt.subplots(1, 2, figsize=(12, 12))

predicted_image_revert = (predicted_image * 127.5 + 127.5).astype(np.uint8)

ax[0].imshow(real_image)
ax[1].imshow(predicted_image_revert)
ax[0].set_title("Input Photo")
ax[1].set_title("Monet-esque")
ax[0].axis("off")
ax[1].axis("off")
plt.show()

It definitely looks different from our input image. If you are unhappy with your results, consider training the model
further - GANs are rather infamous for exhibiting convergence issues and can take a long time to train sufficiently.

## Summary

That's it for this tutorial! By tuning some hyperparameters and training for more
epochs, you can also create high quality Monet renderings of real images. When
you are successful, you immediately know the best configuration because it is all
logged by the immutability of the pipeline! That way, you can immediately share your
YAML configuration file to other people that can reproduce the results.

We hope you had fun with this tutorial, and see you for the next one!